In this file i just made simple manupilations of sample notebook given

In [1]:
import pandas as pd
import numpy as np

# Load the training data (You might have to change the file path based on how you're working)
train_df = pd.read_csv('BH25/Training_Data/train.csv')

print(train_df.head())

   video_id element  motion power  speed          video_summary
0         1    Erde  linear   rot    9.6     (3.8147, 31.94809)
1         2   Feuer     shm  grin    9.6   (26.70288, -4.29153)
2         3   Feuer  random  geld    9.6      (3.8147, 8.58307)
3         4    Erde  zigzag  lila    6.9  (-24.79553, -0.95367)
4         5    Erde  linear  lila    9.6    (7.62939, 22.88818)


In [2]:
#applying filter to the power col as it has noice in the form of spelling mis
replacements = {
    'slau': 'blau', 'baau': 'blau', 'blru': 'blau', 'plau' : 'blau', 'btau' : 'blau',
    'trun' : 'grun', 'brun' : 'grun', 'gaun': 'grun', 'grin':'grun', 'grus' : 'grun',
    'helb' : 'gelb', 'gela' : 'gelb', 'geld' : 'gelb', 'gele' : 'gelb', 'gelt' : 'gelb',
    'rut' : 'rot', 'got' : 'rot', 'not' : 'rot', 'rat' : 'rot', 'rod' : 'rot',
    'lela' : 'lila', 'liga' : 'lila', 'lili' : 'lila', 'lisa' : 'lila', 'lula' : 'lila'
}
train_df['power'] = train_df['power'].replace(replacements)
#finally there are 5 powers
unique_elements2 =train_df['power'].unique()
print(unique_elements2)

['rot' 'grun' 'gelb' 'lila' 'blau']


In [3]:
from sklearn.preprocessing import LabelEncoder

# Encode categorical attributes using the Label Encoder
def encode_categorical_columns(df, columns):
    label_encoders = {}
    for col in columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le
    return df, label_encoders

# List of categorical columns to encode
categorical_columns = ['element', 'motion', 'power']

# Encoding the dataframe
train_df, label_encoders = encode_categorical_columns(train_df, categorical_columns)

# Print the mapping for each column
for col in categorical_columns:
    print(f"Column: {col}")
    print("Attribute -> Encoded Value")
    for attribute, value in zip(label_encoders[col].classes_, range(len(label_encoders[col].classes_))):
        print(f"{attribute} -> {value}")
    print()


Column: element
Attribute -> Encoded Value
Boden -> 0
Erde -> 1
Feuer -> 2
Luft -> 3
Wasser -> 4

Column: motion
Attribute -> Encoded Value
circular -> 0
linear -> 1
random -> 2
shm -> 3
zigzag -> 4

Column: power
Attribute -> Encoded Value
blau -> 0
gelb -> 1
grun -> 2
lila -> 3
rot -> 4



In [4]:
import cv2 
#preprocessing video frame by fram then converting to a numpy array
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (64, 64))
        frames.append(frame)
    cap.release()
    frames = np.array(frames)  # Shape: (20, 64, 64, 3)
    return frames.flatten()  # Flatten to 1D array
    
video_features = []
for idx, row in train_df.iterrows():
    video_path = f"BH25/Training_Data/Train_Videos/{row['video_id']}.mp4"
    video_features.append(process_video(video_path))
#variabe assigned to numpy array of video
video_features = np.array(video_features)  # Shape: (num_samples, 20*64*64*3)


In [5]:
from sklearn.model_selection import train_test_split

#allocating variables to be trained on
y_speed = train_df[['speed']]
y_summary = train_df['video_summary'].apply(lambda x: eval(x))  # Convert string tuples to actual tuples

y_categorical = train_df[categorical_columns]

# Split data for training and validation
X_train_speed, X_val_speed, y_train_speed, y_val_speed = train_test_split(video_features, y_speed, test_size=0.2, random_state=42)
X_train_cat, X_val_cat, y_train_cat, y_val_cat = train_test_split(video_features, y_categorical, test_size=0.2, random_state=42)
X_train_summary, X_val_summary, y_train_summary, y_val_summary = train_test_split(video_features, y_summary, test_size=0.2, random_state=42)

In [6]:
#processing data in batchs for speed cat video summary
X_batch_speed=X_train_speed[:3000]
y_batch_speed=y_train_speed[:3000]

In [32]:
print("Shape of X_batch_speed:", X_batch_speed.shape)
print("Shape of y_batch_speed:", y_batch_speed.shape)

Shape of X_batch_speed: (1000, 245760)
Shape of y_batch_speed: (1000, 1)


In [7]:
X_batch_cat = X_train_cat[:1000]
y_batch_cat = y_train_cat[:1000]

In [8]:
X_batch_summary = X_train_summary[:2000]
y_batch_summary = y_train_summary[:2000]

In [50]:
# Train continuous target model
from sklearn.linear_model import LinearRegression
model_continuous = LinearRegression()
model_continuous.fit(X_batch_speed, y_batch_speed)

LinearRegression()

In [52]:
#for linear model mae value
from sklearn.metrics import mean_absolute_error

# Predict the continuous labels for the validation set
y_val_pred_continuous = model_continuous.predict(X_val_speed)

# Calculate and print the Mean Absolute Error
mae_continuous = mean_absolute_error(y_val_speed, y_val_pred_continuous)
print(f'Mean Absolute Error: {mae_continuous}')


Mean Absolute Error: 0.5371062096347837


In [10]:
# Train models for categorical columns using RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
models_categorical = {}
for col in categorical_columns:
    
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_batch_cat, y_batch_cat[col])
    models_categorical[col] = model

In [11]:
#not using random forest for element col 
from sklearn.linear_model import LogisticRegression
models_categorical['element'] = LogisticRegression(multi_class='multinomial', solver='lbfgs',max_iter =1000)
models_categorical['element'].fit(X_batch_cat,y_batch_cat['element'])

c:\Users\hp\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='multinomial')

In [23]:
# Training for video_summary, separate models for x and y here using linear regression model
from sklearn.linear_model import LogisticRegression
x_values, y_values = zip(*y_batch_summary)

model_summary_x = LinearRegression()
model_summary_x.fit(X_batch_summary, x_values)

model_summary_y = LinearRegression()
model_summary_y.fit(X_batch_summary, y_values)

LinearRegression()

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, mean_absolute_error
y_speed_pred = model_continuous.predict(X_val_speed)
mae_continuous = mean_absolute_error(y_val_speed, y_speed_pred)

categorical_accuracy = {}
categorical_classification_report = {}
categorical_confusion_matrix = {}
for col in categorical_columns:
        y_categorical_pred = models_categorical[col].predict(X_val_cat)
        accuracy = accuracy_score(y_val_cat[col],y_categorical_pred)
        classification_reportvar = classification_report(y_val_cat[col], y_categorical_pred)
        confusion_matrixvvar = confusion_matrix(y_val_cat[col], y_categorical_pred)
        categorical_accuracy[col] = accuracy
        categorical_classification_report[col] = classification_reportvar
        categorical_confusion_matrix[col] = confusion_matrixvvar

x_val_values, y_val_values = zip(*y_val_summary)


x_values_pred = model_summary_x.predict(X_val_summary)
mae_summary_x = mean_absolute_error(x_val_values, x_values_pred)


y_values_pred = model_summary_y.predict(X_val_summary)
mae_summary_y = mean_absolute_error(y_val_values, y_values_pred)

In [18]:
print(mae_summary_x)
print(mae_summary_y)
print(categorical_accuracy)
print(mae_continuous)

{'element': 0.7175, 'motion': 0.9985, 'power': 0.9745}
0.5371062096347837


In [19]:
import os

test_folder = 'BH25/Testing_Data'

test_features = []
video_ids = []

# The below function finds all the video ids in the test folder and sorts them, 
# and then stores their features in test_features

for video_id in sorted(
    [f for f in os.listdir(test_folder) if f.endswith(".mp4")], key=lambda x: int(x.split('.')[0])):
    video_path = os.path.join(test_folder, video_id)
    video_ids.append(video_id.split('.')[0]) 
    test_features.append(process_video(video_path))

test_features = np.array(test_features)  # Shape: (num_test_samples, 20*64*64*3)

In [25]:
y_test_speed = model_continuous.predict(test_features)
y_test_summary_x = model_summary_x.predict(test_features)
y_test_summary_y = model_summary_y.predict(test_features)

In [26]:
y_categorical = {}

for col in categorical_columns:
    predictions = models_categorical[col].predict(test_features)
    y_categorical[col] =label_encoders[col].inverse_transform(predictions)

In [59]:
speed_predictions = y_test_speed.flatten()  # Ensure the shape is (num_samples,)
summary_x_predictions = y_test_summary_x.flatten()
summary_y_predictions = y_test_summary_y.flatten()

In [60]:
output_df = pd.DataFrame({
    'video_id': video_ids,                      
    'element': y_categorical['element'],  
    'motion': y_categorical['motion'],
    'power': y_categorical['power'],
    'speed': speed_predictions,               
    'video_summary': list(zip(summary_x_predictions, summary_y_predictions)) 
})

In [62]:
output_df.to_csv("speed.csv", index=False)

# This is the final submission file to be submitted